Ziad Amr Salah 1190103
Ramy Ashraf    1190178

spice Circuit simulation engine

inputfile shape

    Netlist Format:
    Simulation_time
    # of iterations
    -Component 1
    -Component2
    …
    -1 (indicates end of input)
    Each component has the following format:
    ------------------------------------------------------------------------------
    Component_Type | Node1 | Node2 | Value | Initial_Value
    ------------------------------------------------------------------------------

    - Component_Type stands for the type of the component. It can be one of the following

In [124]:
import numpy as np
Vsrc_order = 1
results = dict()

CapOrIndExist = False  # should be changed to true if capacitor exists
filename = "testcases/6.txt"

In [125]:
class Resistor:
    def __init__(self, value, Node1, Node2):
        self.value = 1/value
        self.n1 = Node1
        self.n2 = Node2

    def add(self, G, N):
        rStamp = np.zeros((N, N))
        if (self.n1 == 0):
            rStamp[self.n2-1, self.n2-1] = self.value
        elif (self.n2 == 0):
            rStamp[self.n1-1, self.n1-1] = self.value
        else:
            rStamp[self.n1-1, self.n1-1] = self.value
            rStamp[self.n2-1, self.n2-1] = self.value
            rStamp[self.n1-1, self.n2-1] = (-1)*self.value
            rStamp[self.n2-1, self.n1-1] = (-1)*self.value
        G = G+rStamp
        return G


In [126]:
class Capacitor:
    def __init__(self, value, initial_value, Node1, Node2):
        self.value = value
        self.initial_value = initial_value
        self.n1 = Node1
        self.n2 = Node2

    def add(self, G, I, N, current_H, current_iteration, v_n1, v_n2):
        cStamp = np.zeros((N, N))
        icStamp = np.zeros((N, 1))
        # negative node on ground
        if self.n2 == 0:

            if current_iteration == 0:
                cStamp[self.n1-1, self.n1-1] = self.value/current_H
                icStamp[self.n1-1, 0] = (self.value /
                                         current_H)*self.initial_value
            else:
                icStamp[self.n1-1, 0] = (self.value/current_H)*(v_n2)

        # positive node on ground
        elif self.n1 == 0:
            if current_iteration == 0:
                cStamp[self.n2-1, self.n2-1] = self.value/current_H

                icStamp[self.n2-1, 0] = (self.value /
                                         current_H)*self.initial_value
            else:
                icStamp[self.n2-1, 0] = (self.value/current_H)*(v_n1)

        # no node is on ground
        else:

            if current_iteration == 0:
                cStamp[self.n1-1, self.n1-1] = self.value/current_H
                cStamp[self.n2-1, self.n2-1] = self.value/current_H
                cStamp[self.n1-1, self.n2-1] = -1*(self.value/current_H)
                cStamp[self.n2-1, self.n1-1] = -1*(self.value/current_H)
                icStamp[self.n1-1, 0] = (self.value /
                                         current_H)*self.initial_value
                icStamp[self.n2-1, 0] = (self.value /
                                         current_H)*self.initial_value*-1
            else:
                icStamp[self.n1-1, 0] = (self.value/current_H)*(v_n2-v_n1)
                icStamp[self.n2-1, 0] = (self.value/current_H)*(v_n2-v_n1)*-1
        G = G+cStamp
        if self.n1 == 0:
            I[self.n2-1, 0] = icStamp[self.n2-1, 0]
        else:
            I[self.n1-1, 0] = icStamp[self.n1-1, 0]

        return G, I


In [127]:
class Inductor:
    def __init__(self, value, initial_value, Node1, Node2):
        global Vsrc_order
        self.i = Vsrc_order
        Vsrc_order += 1
        self.value = value
        self.n1 = Node1
        self.n2 = Node2
        self.initial_value = initial_value

    def add(self, B, C, D, E, current_H, current_iteration, i_v1, i_v2, M, N):
        bStamp = np.zeros((N, M))
        eStamp = np.zeros((M, 1))
        dStamp = np.zeros((M, M))
        # negative on ground

        if self.n2 == 0:
            if current_iteration == 0:
                bStamp[self.n1-1, self.i-1] = 1
                eStamp[self.i-1, 0] = -1 * \
                    (self.value/current_H)*(self.initial_value)
                dStamp[self.i-1, self.i-1] = - \
                    1*(self.value/current_H)
            else:
                eStamp[self.i-1, 0] = -1*(self.value/current_H)*(i_v2)

        # positive on ground
        elif self.n1 == 0:
            if current_iteration == 0:
                bStamp[self.n2-1, self.i-1] = 1
                eStamp[self.i-1, 0] = -1 * \
                    (self.value/current_H)*(self.initial_value)
                dStamp[self.i-1, self.i-1] = - \
                    1*(self.value/current_H)

            else:
                eStamp[self.i-1, 0] = -1*(self.value/current_H)*(i_v1)

        # no port on ground
        else:
            if current_iteration == 0:
                bStamp[self.n1-1, self.i-1] = 1
                bStamp[self.n2-1, self.i-1] = -1
                eStamp[self.i-1, 0] = -1 * \
                    (self.value/current_H)*(self.initial_value)
                dStamp[self.i-1, self.i-1] = - \
                    1*(self.value/current_H)

            else:
                eStamp[self.i-1, 0] = -1*(self.value/current_H)*(i_v1)
        B = B+bStamp
        C = C+np.transpose(bStamp)
        D = D+dStamp
        E[self.i-1, 0] = eStamp[self.i-1, 0]
        return B, C, D, E

In [128]:
class VoltageSrc:
    def __init__(self, value, initial_value, Node1, Node2, m):
        self.value = value
        self.initial_value = initial_value
        self.n1 = Node1
        self.n2 = Node2
        self.i = m

    def add(self, B, C, E, N, M):
        vStamp = np.zeros((N, M))
        eStamp = np.zeros((M, 1))
        if (self.n1 == 0):
            vStamp[self.n2-1, self.i-1] = -1
            eStamp[self.i-1, 0] = self.value
        elif (self.n2 == 0):
            vStamp[self.n1-1, self.i-1] = 1
            eStamp[self.i-1, 0] = self.value
        else:
            vStamp[self.n1-1, self.i-1] = 1
            vStamp[self.n2-1, self.i-1] = -1
            eStamp[self.i-1, 0] = self.value
        B = B+vStamp
        C = C+np.transpose(vStamp)
        E = E+eStamp
        return B, C, E


class CurrentSrc:
    def __init__(self, value, initial_value, Node1, Node2):
        self.value = value
        self.initial_value = initial_value
        self.n1 = Node1
        self.n2 = Node2

    def add(self, I, N):
        iStamp = np.zeros((N, 1))
        if (self.n1 == 0):
            iStamp[self.n2-1, 0] = self.value
        elif (self.n2 == 0):
            iStamp[self.n1-1, 0] = self.value
        else:
            iStamp[self.n1-1, 0] = self.value
            iStamp[self.n2-1, 0] = self.value * (-1)
        I = I+iStamp
        return I

In [129]:
# added read from file
file = open(filename, 'r')
lines = list(file.readlines())
for i in range(0, len(lines)):
    lines[i] = lines[i].replace('\n', '')
# remove the last line
lines.pop()


Res_arr = list()
Vsrc_arr = list()
Isrc_arr = list()
Cap_arr = list()
Ind_arr = list()


nodes = set()

step_size = float(lines[0])
iterations = int(lines[1])

for i in range(2, len(lines)):
    details = lines[i].split(' ')
    type_of_component = details[0]
    # resistor
    if type_of_component == 'R':

        Node1 = int(details[1].replace('V', ''))
        Node2 = int(details[2].replace('V', ''))
        nodes.add(Node1)
        nodes.add(Node2)
        Value = float(details[3])
        Initial_value = float(details[4])

        R = Resistor(Value, Node1, Node2,)
        Res_arr.append(R)

    # Voltage src

    elif type_of_component == 'Vsrc':

        Node1 = int(details[1].replace('V', ''))
        Node2 = int(details[2].replace('V', ''))
        nodes.add(Node1)
        nodes.add(Node2)
        Value = float(details[3])
        Initial_value = float(details[4])

        V = VoltageSrc(Value, Initial_value, Node1, Node2, Vsrc_order)
        Vsrc_order += 1
        Vsrc_arr.append(V)
        
    # current src
    elif type_of_component == 'Isrc':
        Node1 = int(details[1].replace('V', ''))
        Node2 = int(details[2].replace('V', ''))
        nodes.add(Node1)
        nodes.add(Node2)
        Value = float(details[3])
        Initial_value = float(details[4])
        I = CurrentSrc(Value, Initial_value, Node1, Node2)
        Isrc_arr.append(I)
        
    # capacitor
    elif type_of_component == 'C':

        Node1 = int(details[1].replace('V', ''))
        Node2 = int(details[2].replace('V', ''))
        nodes.add(Node1)
        nodes.add(Node2)
        Value = float(details[3])
        Initial_value = float(details[4])
        CapOrIndExist = True
        C = Capacitor(Value, Initial_value, Node1, Node2)
        Cap_arr.append(C)

    # inductor
    elif type_of_component == 'I':

        Node1 = int(details[1].replace('V', ''))
        Node2 = int(details[2].replace('V', ''))
        nodes.add(Node1)
        nodes.add(Node2)
        Value = float(details[3])
        Initial_value = float(details[4])
        CapOrIndExist = True
        I = Inductor(Value, Initial_value, Node1, Node2)
        Ind_arr.append(I)
        


In [130]:
# number of voltage sources
m = len(Vsrc_arr)+len(Ind_arr)


# number of nodes
nodes = list(nodes)
n = max(nodes)
# Initializing matrices

# matrix A components
matrixG = np.zeros((n, n))
matrixB = np.zeros((n, m))
matrixC = np.zeros((m, n))
matrixD = np.zeros((m, m))


# matrix Z components


matrixI = np.zeros((n, 1))
matrixE = np.zeros((m, 1))


matrixX = np.empty((n+m, 1))

# loop for resistors
for i in range(0, len(Res_arr)):
    matrixG = Res_arr[i].add(matrixG, n)

# loop for Vsrcs
for i in range(0, len(Vsrc_arr)):
    [matrixB, matrixC, matrixE] = Vsrc_arr[i].add(
        matrixB, matrixC, matrixE, n, m)

# loop for Isrcs
for i in range(0, len(Isrc_arr)):
    matrixI = Isrc_arr[i].add(matrixI, n)

# loop for Cap
for i in range(0, len(Cap_arr)):
    [matrixG, matrixI] = Cap_arr[i].add(
        matrixG, matrixI, n, step_size, 0, 0, 0)

# loop for inductors
for i in range(0, len(Ind_arr)):
    [matrixB, matrixC, matrixD, matrixE] = Ind_arr[i].add(
        matrixB, matrixC, matrixD, matrixE, step_size, 0, 0, 0, m, n)

# print(matrixG)
matrixA = np.concatenate((np.concatenate(
    (matrixG, matrixB), axis=1), np.concatenate((matrixC, matrixD), axis=1)), axis=0)
# print(matrixA)
matrixZ = np.concatenate((matrixI, matrixE))
# print(matrxiZ)
matrixX = np.matmul(np.linalg.inv(matrixA), matrixZ)
print(matrixX)

[[15.]
 [10.]
 [-5.]
 [ 5.]]


In [131]:

voltage_node_count = 0
current_from_sources_count = 0
current_from_inductor_count = 0

for i in range(0, m+n):
    
    # voltage at nodes
    if i < n:
        key_string = 'V'+str(voltage_node_count)
        voltage_node_count += 1
        

    # currents from voltage sources
    elif i < n+len(Vsrc_arr):
        key_string = 'I_Vsrc'+str(current_from_sources_count)
        current_from_inductor_count += 1
        

    # currents from inductors
    else:
        key_string = 'I_L'+str(current_from_inductor_count)
        current_from_inductor_count += 1
        
    results[key_string + " iteration: 0 " ] =  str(matrixX[i, 0]) 


if CapOrIndExist == True:
    iteration_count = 1
    for i in range(1, iterations):
        # capacitor loop
        for j in range(0, len(Cap_arr)):
            [matrixG, matrixI] = Cap_arr[j].add(
                matrixG, matrixI, n, step_size, i, matrixX[Cap_arr[j].n2 - 1, 0], matrixX[Cap_arr[j].n1-1, 0])
        # inductor loop
        for j in range(0, len(Ind_arr)):
            [matrixB, matrixC, matrixD, matrixE] = Ind_arr[j].add(
                matrixB, matrixC, matrixD, matrixE, step_size, i, matrixX[Ind_arr[j].i+n-1, 0], matrixX[Ind_arr[j].i+n-1, 0], m, n)

        matrixA = np.concatenate((np.concatenate(
            (matrixG, matrixB), axis=1), np.concatenate((matrixC, matrixD), axis=1)), axis=0)


        matrixZ = np.concatenate((matrixI, matrixE))
        matrixX = np.matmul(np.linalg.inv(matrixA), matrixZ)
        # print(matrixA)
        #print(matrixX)
        
        
        iteration_count += 1
        for i in range(0, m+n):
            if i < n:
                key_string = 'V'+str(i)

        # currents from voltage sources
            elif i < n+len(Vsrc_arr):
                key_string = 'I_Vsrc'+str(i-n)

        # currents from inductors
            else:
                key_string = 'I_L'+str(i-n-len(Vsrc_arr))
                
            results[key_string + " iteration: " + str(iteration_count -1) ] =  str(matrixX[i, 0]) 
            


print("done")


done


In [132]:
print(matrixA)

[[ 1. -1.  1.  0.]
 [-1.  1.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0. -2.]]


In [133]:
print(matrixX)

[[ 1.50000000e+01]
 [ 4.51092990e-03]
 [-1.49954891e+01]
 [ 1.49954891e+01]]


In [134]:
keys = list(results.keys())
values = list(results.values())
n = len(keys) // 3 # replace with the desired number of loops

for i in range(n):
    print(keys[i], " ", values[i])

V0 iteration: 0    15.0
V1 iteration: 0    10.000000000000002
I_Vsrc0 iteration: 0    -5.0
I_L1 iteration: 0    5.0
V0 iteration: 1   15.0
V1 iteration: 1   6.666666666666669
I_Vsrc0 iteration: 1   -8.333333333333332
I_L0 iteration: 1   8.333333333333332
V0 iteration: 2   15.0
V1 iteration: 2   4.444444444444447
I_Vsrc0 iteration: 2   -10.555555555555554
I_L0 iteration: 2   10.555555555555554
V0 iteration: 3   15.0
V1 iteration: 3   2.9629629629629664
I_Vsrc0 iteration: 3   -12.037037037037035
I_L0 iteration: 3   12.037037037037035
V0 iteration: 4   15.0
V1 iteration: 4   1.975308641975312
I_Vsrc0 iteration: 4   -13.02469135802469
I_L0 iteration: 4   13.02469135802469
V0 iteration: 5   15.0
V1 iteration: 5   1.316872427983542
I_Vsrc0 iteration: 5   -13.68312757201646
I_L0 iteration: 5   13.68312757201646
V0 iteration: 6   15.0
V1 iteration: 6   0.8779149519890286


In [135]:
for i in range(n, 2*n):
    print(keys[i], " ", values[i])

I_Vsrc0 iteration: 6   -14.122085048010973
I_L0 iteration: 6   14.122085048010973
V0 iteration: 7   15.0
V1 iteration: 7   0.585276634659353
I_Vsrc0 iteration: 7   -14.414723365340649
I_L0 iteration: 7   14.414723365340649
V0 iteration: 8   15.0
V1 iteration: 8   0.3901844231062359
I_Vsrc0 iteration: 8   -14.609815576893766
I_L0 iteration: 8   14.609815576893766
V0 iteration: 9   15.0
V1 iteration: 9   0.2601229487374912
I_Vsrc0 iteration: 9   -14.73987705126251
I_L0 iteration: 9   14.73987705126251
V0 iteration: 10   15.0
V1 iteration: 10   0.17341529915832865
I_Vsrc0 iteration: 10   -14.826584700841673
I_L0 iteration: 10   14.826584700841673
V0 iteration: 11   15.0
V1 iteration: 11   0.11561019943888695
I_Vsrc0 iteration: 11   -14.884389800561115
I_L0 iteration: 11   14.884389800561115
V0 iteration: 12   15.0
V1 iteration: 12   0.07707346629259249
I_Vsrc0 iteration: 12   -14.92292653370741
I_L0 iteration: 12   14.92292653370741


In [136]:
for i in range(2*n, 3*n):
    print(keys[i], " ", values[i])

V0 iteration: 13   15.0
V1 iteration: 13   0.05138231086172951
I_Vsrc0 iteration: 13   -14.948617689138272
I_L0 iteration: 13   14.948617689138272
V0 iteration: 14   15.0
V1 iteration: 14   0.03425487390782145
I_Vsrc0 iteration: 14   -14.96574512609218
I_L0 iteration: 14   14.96574512609218
V0 iteration: 15   15.0
V1 iteration: 15   0.022836582605215483
I_Vsrc0 iteration: 15   -14.977163417394786
I_L0 iteration: 15   14.977163417394786
V0 iteration: 16   15.0
V1 iteration: 16   0.015224388403478173
I_Vsrc0 iteration: 16   -14.984775611596524
I_L0 iteration: 16   14.984775611596524
V0 iteration: 17   15.0
V1 iteration: 17   0.01014959226898604
I_Vsrc0 iteration: 17   -14.989850407731016
I_L0 iteration: 17   14.989850407731016
V0 iteration: 18   15.0
V1 iteration: 18   0.006766394845991286
I_Vsrc0 iteration: 18   -14.99323360515401
I_L0 iteration: 18   14.99323360515401
V0 iteration: 19   15.0
V1 iteration: 19   0.004510929897328708
